In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import os


In [11]:
# Define data transformations for data augmentation and normalization
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

"""
All pre-trained models expect input images normalized in the same way, i.e. mini-batches of 3-channel RGB images of shape (3 x H x W), where H and W are expected to be at least 224. 
The images have to be loaded in to a range of [0, 1] and then normalized using mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225].
"""


'\nAll pre-trained models expect input images normalized in the same way, i.e. mini-batches of 3-channel RGB images of shape (3 x H x W), where H and W are expected to be at least 224. \nThe images have to be loaded in to a range of [0, 1] and then normalized using mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225].\n'

In [12]:
# Define the data directory
data_dir = 'dataset'

# Create data loaders
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
#image_datasets

In [13]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
print(dataset_sizes)

class_names = image_datasets['train'].classes
class_names

{'train': 4411, 'val': 490}


['alstonia_scholaris_diseased',
 'alstonia_scholaris_healthy',
 'arjun_diseased',
 'arjun_healthy',
 'bael_diseased',
 'basil_healthy',
 'chinar_diseased',
 'chinar_healthy',
 'gauva_diseased',
 'gauva_healthy',
 'jamun_diseased',
 'jamun_healthy',
 'jatropha_diseased',
 'jatropha_healthy',
 'lemon_diseased',
 'lemon_healthy',
 'mango_diseased',
 'mango_healthy',
 'pomegranate_diseased',
 'pomegranate_healthy',
 'pongamia_pinnata_diseased',
 'pongamia_pinnata_healthy']

In [14]:
# Load the pre-trained ResNet-18 model
model = models.resnet18(pretrained=True)

# Freeze all layers except the final classification layer
for name, param in model.named_parameters():
    if "fc" in name:  # Unfreeze the final classification layer
        param.requires_grad = True
    else:
        param.requires_grad = False

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)  # Use all parameters


# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)


cuda:0


In [34]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')


print("Training complete!")


train Loss: 2.2260 Acc: 0.3954
val Loss: 0.7381 Acc: 0.7633
train Loss: 1.2603 Acc: 0.6069
val Loss: 0.5384 Acc: 0.8245
train Loss: 1.0543 Acc: 0.6611
val Loss: 0.4053 Acc: 0.8735
train Loss: 0.9846 Acc: 0.6862
val Loss: 0.4014 Acc: 0.8694
train Loss: 0.9193 Acc: 0.7057
val Loss: 0.4553 Acc: 0.8592
train Loss: 0.8949 Acc: 0.7155
val Loss: 0.3632 Acc: 0.8755
train Loss: 0.8599 Acc: 0.7316
val Loss: 0.3380 Acc: 0.8939
train Loss: 0.8603 Acc: 0.7309
val Loss: 0.2747 Acc: 0.9020
train Loss: 0.8484 Acc: 0.7311
val Loss: 0.3109 Acc: 0.8959
train Loss: 0.8168 Acc: 0.7325
val Loss: 0.3406 Acc: 0.8918
Training complete!


In [35]:

# Save the model
torch.save(model.state_dict(), '30E.pth')


# Classification on Unseen Image

To use the saved model to classify unseen images, you need to load the model and then apply it to the new images for inference. 

In [6]:
import torch
from torchvision import models, transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import os

# Load the saved model
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 1000)  # Adjust to match the original model's output units
model.load_state_dict(torch.load('50E.pth'))
model.eval()

# Create a new model with the correct final layer
new_model = models.resnet18(pretrained=True)
new_model.fc = nn.Linear(new_model.fc.in_features, 2)  # Adjust to match the desired output units

# Copy the weights and biases from the loaded model to the new model
new_model.fc.weight.data = model.fc.weight.data[0:2]  # Copy only the first 2 output units
new_model.fc.bias.data = model.fc.bias.data[0:2]

# Map the predicted class to the class name
class_names = ['alstonia_scholaris_diseased',
 'alstonia_scholaris_healthy',
 'arjun_diseased',
 'arjun_healthy',
 'bael_diseased',
 'basil_healthy',
 'chinar_diseased',
 'chinar_healthy',
 'gauva_diseased',
 'gauva_healthy',
 'jamun_diseased',
 'jamun_healthy',
 'jatropha_diseased',
 'jatropha_healthy',
 'lemon_diseased',
 'lemon_healthy',
 'mango_diseased',
 'mango_healthy',
 'pomegranate_diseased',
 'pomegranate_healthy',
 'pongamia_pinnata_diseased',
 'pongamia_pinnata_healthy']

Prepare your new image for classification. You should use the same data transformations you used during training. Here's an example of how to prepare an image for inference:

In [7]:
folders = os.listdir("dataset/test")

SHOWIMAGE = False

counter = 0
correct = 0
correct_leaf = 0
wrong = 0
wrong_leaf = 0

for folder in folders:
    files = os.listdir("dataset/test/"+folder)
    print(f'{folder}')
    print(f'{files=}')
    print(f'{len(files)=}')
    for file in files:
        # Load and preprocess the unseen image
        image_path = "dataset/test/"+folder+"/"+file  # Replace with the path to your image
        image = Image.open(image_path)
        preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        input_tensor = preprocess(image)
        input_batch = input_tensor.unsqueeze(0)  # Add a batch dimension


        #Perform inference using the model:
        with torch.no_grad():
            output = model(input_batch)

        # Get the predicted class
        _, predicted_class = output.max(1)

        predicted_class_name = class_names[predicted_class.item()]

        counter += 1
        if predicted_class_name == folder:
            correct += 1
            #print(f'The predicted class is: {predicted_class_name}')
        else:
            wrong += 1
            print(f'The predicted class is: {predicted_class_name} should be {folder}')
            
        if predicted_class_name.replace('_healthy', '').replace('_diseased', '') == folder.replace('_healthy', '').replace('_diseased', ''):
            correct_leaf += 1
            #print(f'The predicted class is: {predicted_class_name}')
        else:
            wrong_leaf += 1
        
        if SHOWIMAGE:
            # Display the image with the predicted class name
            image = np.array(image)
            plt.imshow(image)
            plt.axis('off')
            plt.text(10, 10, f'Predicted: {predicted_class_name}', fontsize=12, color='white', backgroundcolor='red')
            plt.show()
        
    
print(f'{counter = }')
print(f'{correct = }')
print(f'{wrong = }')
print(f'{correct_leaf = }')
print(f'{wrong_leaf = }')

files=['0014_0230.JPG', '0014_0231.JPG', '0014_0232.JPG', '0014_0233.JPG', '0014_0234.JPG', '0014_0235.JPG', '0014_0236.JPG', '0014_0237.JPG', '0014_0238.JPG', '0014_0239.JPG', '0014_0240.JPG', '0014_0241.JPG', '0014_0242.JPG', '0014_0243.JPG', '0014_0244.JPG', '0014_0245.JPG', '0014_0246.JPG', '0014_0247.JPG', '0014_0248.JPG', '0014_0249.JPG', '0014_0250.JPG', '0014_0251.JPG', '0014_0252.JPG', '0014_0253.JPG', '0014_0254.JPG']
len(files)=25
The predicted class is: arjun_healthy should be alstonia_scholaris_diseased
The predicted class is: arjun_diseased should be alstonia_scholaris_diseased
The predicted class is: alstonia_scholaris_healthy should be alstonia_scholaris_diseased
The predicted class is: alstonia_scholaris_healthy should be alstonia_scholaris_diseased
The predicted class is: alstonia_scholaris_healthy should be alstonia_scholaris_diseased
files=['0003_0162.JPG', '0003_0163.JPG', '0003_0164.JPG', '0003_0165.JPG', '0003_0166.JPG', '0003_0167.JPG', '0003_0168.JPG', '0003_01

In [8]:
# import matplotlib.pyplot as plt
# import numpy
# from sklearn import metrics

# actual = numpy.random.binomial(1,.9,size = 1000)
# predicted = numpy.random.binomial(1,.9,size = 1000)

# confusion_matrix = metrics.confusion_matrix(actual, predicted)

# cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

# cm_display.plot()
# plt.show()

pth - Class   | Leaf
10E - 400, 70 | 442, 28 
50E - 423, 47 | 453, 17